In [15]:
from project_lib import Project
project = Project()
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

In [17]:
from json import loads
from IPython.display import JSON

In [18]:
%env USERNAME=admin
%env PASSWORD=uoKsHWi3obej
%env DATAPLATFORM_URL=https://cpd-wml.apps.neptune.cp.fyre.ibm.com

%env SPACE_ID=7b648ffc-53e3-4b16-8e59-3c55ef755f31
#db947328-7845-4f4b-a87c-2ca721960d74



env: USERNAME=admin
env: PASSWORD=uoKsHWi3obej
env: DATAPLATFORM_URL=https://cpd-wml.apps.neptune.cp.fyre.ibm.com
env: SPACE_ID=7b648ffc-53e3-4b16-8e59-3c55ef755f31


In [19]:
%env SPACE_ID  
#new space

'7b648ffc-53e3-4b16-8e59-3c55ef755f31'

In [20]:
wml_credentials = {
    "username": "admin",
    "passowrd": "uoKsHWi3obej",
    "url": "https://cpd-wml.apps.neptune.cp.fyre.ibm.com",
    "instance_id": 'openshift',
    "version": '4.7'
}

In [21]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

client.set.default_space("1ad236d9-2b4d-4853-826f-358f58212ad2")
#db947328-7845-4f4b-a87c-2ca721960d74")

'SUCCESS'

#### get auth token

In [22]:
%%bash --out token

token=$(curl -sk -X POST \
        --header "Content-type: application/json" \
        -d "{\"username\":\"${USERNAME}\",\"password\":\"${PASSWORD}\"}" \
        "$DATAPLATFORM_URL/icp4d-api/v1/authorize")

token=${token#*token\":\"}
token=${token%%\"*}
echo $token

In [10]:
%env TOKEN=$token

env: TOKEN=eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImVZa2h5dGxMelFBcGt4ZUFxR3NIOGVaYTA3aXhqMXlzcVJzYTJLdnlHdncifQ.eyJ1c2VybmFtZSI6ImFkbWluIiwicm9sZSI6IkFkbWluIiwicGVybWlzc2lvbnMiOlsiY3JlYXRlX3Byb2plY3QiLCJtYW5hZ2VfY2F0YWxvZyIsIm1hbmFnZV92YXVsdHNfYW5kX3NlY3JldHMiLCJzaGFyZV9zZWNyZXRzIiwiYWRkX3ZhdWx0cyIsImFkbWluaXN0cmF0b3IiLCJjYW5fcHJvdmlzaW9uIiwibW9uaXRvcl9wbGF0Zm9ybSIsImNvbmZpZ3VyZV9wbGF0Zm9ybSIsInZpZXdfcGxhdGZvcm1faGVhbHRoIiwiY29uZmlndXJlX2F1dGgiLCJtYW5hZ2VfdXNlcnMiLCJtYW5hZ2VfZ3JvdXBzIiwibWFuYWdlX3NlcnZpY2VfaW5zdGFuY2VzIiwiY3JlYXRlX3NwYWNlIiwiYWNjZXNzX2NhdGFsb2ciXSwiZ3JvdXBzIjpbMTAwMDBdLCJzdWIiOiJhZG1pbiIsImlzcyI6IktOT1hTU08iLCJhdWQiOiJEU1giLCJ1aWQiOiIxMDAwMzMwOTk5IiwiYXV0aGVudGljYXRvciI6ImRlZmF1bHQiLCJkaXNwbGF5X25hbWUiOiJhZG1pbiIsImFwaV9yZXF1ZXN0IjpmYWxzZSwiaWF0IjoxNjk4NzI1Mzg0LCJleHAiOjE2OTg3Njg1ODR9.bYfatLE9Nqr7FMuYYW2TXxUlQ5Y444HXCiSF0vmXe0nQZJZxelwaDLxwPCagU_ck5MIi51jwGBGbH3Ae_wjMj6pcy1Nyba-T2XF8tQs3I6-2iEXe1vasGzcSDO-5YqSfYKg0Ck4W2D_BijyAogHrrT4vqjFZy0QjsnIdOUxvU93wHf4ZskGdF

## Python Function

#### Define the function

In [433]:
## wrapper function 
def deployable_callable():
    """
    Deployable python function with score
    function implemented.
    """
    #try:
        #from statsmodels.stats.descriptivestats import describe
    n = 2
    #except ModuleNotFoundError as e:
        #print(f"statsmodels not installed: {str(e)}")
        
    def score(payload):
        """
        Score method.
        """
        try:
            data = payload['input_data'][0]['values']
            #print(data)
            return {
                'predictions': [
                    {'values': [i*n for i in data]}#, describe(data)]}
                ]
            }
        except Exception as e:
            return {'predictions': [{'values': [repr(e)]}]}
        
    return score


In [434]:
import numpy as np

data = np.random.randint(2,20,2)
data_description = deployable_callable()({
    "input_data": [{
        "values" : [data]
    }]
})
print(data)
print(data_description["predictions"][0]["values"])

[2 4]
[array([4, 8])]


#### Create the function - add the function to WML repo 

In [26]:
%%bash --out scoring_payload

SCORING_PAYLOAD='{"name": "scaling python function2", "space_id": "'"$SPACE_ID"'", "software_spec": {"name": "runtime-22.2-py3.10"},"type": "python","sample_scoring_input": {  "input_data": [{ "values" : [12] }]}}'

echo $SCORING_PAYLOAD


In [27]:
JSON(loads(scoring_payload))

<IPython.core.display.JSON object>

In [28]:
%env SCORING_PAYLOAD=$scoring_payload

env: SCORING_PAYLOAD={"name": "scaling python function2", "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31", "software_spec": {"name": "runtime-22.2-py3.10"},"type": "python","sample_scoring_input": { "input_data": [{ "values" : [12] }]}}


In [29]:
%%bash --out function_id -s "$scoring_payload"


curl -sk -X POST  "$DATAPLATFORM_URL/ml/v4/functions?version=2023-10-26"\
        --header "Authorization: Bearer $TOKEN" \
        --header "Content-Type: application/json" \
        --header "Accept: application/json" \
        --data-raw "$SCORING_PAYLOAD" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p
                   

#### get function id

In [30]:
%env FUNCTION_ID=$function_id

env: FUNCTION_ID=bcc0ee29-3430-481a-a694-250b98894b5d


#### Create revision of the function

In [31]:
%%bash --out func_revision_payload

FUNC_REVISION_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "commit_message": "Updating Score Function"}'
echo $FUNC_REVISION_PAYLOAD

In [32]:
%env FUNC_REVISION_PAYLOAD=$func_revision_payload

env: FUNC_REVISION_PAYLOAD={"space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31", "commit_message": "Updating Score Function"}


In [33]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$FUNC_REVISION_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/functions/$FUNCTION_ID/revisions?version=2023-10-26" \
    | python -m json.tool

{
    "entity": {
        "sample_scoring_input": {
            "input_data": [
                {
                    "values": [
                        12
                    ]
                }
            ]
        },
        "software_spec": {
            "id": "b56101f1-309d-549b-a849-eaa63f77b2fb",
            "name": "runtime-22.2-py3.10"
        },
        "type": "python"
    },
    "metadata": {
        "commit_info": {
            "commit_message": "Updating Score Function",
            "committed_at": "2023-10-31T13:14:01.002Z"
        },
        "created_at": "2023-10-31T13:13:52.700Z",
        "id": "bcc0ee29-3430-481a-a694-250b98894b5d",
        "modified_at": "2023-10-31T13:13:52.700Z",
        "name": "scaling python function2",
        "owner": "1000330999",
        "rev": "1",
        "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
    },
    "system": {
        "warnings": []
    }
}


#### Upload the code to scoring function

#### get the score.py.gz from git

In [36]:
!wget -q https://raw.githubusercontent.com/hsanthan/qiskit-machine-learning/main/my_scoring_func.py -O score_fun_git.py

In [37]:
%%bash
cat score_fun_git.py

def score(payload):
        """
        Score method.
        """
        try:
            data = payload['input_data'][0]['values']
            # print(data)
            return {
                'predictions': [
                    {'values': [(data)*2]}  # , describe(data)]}
                ]
            }
        except Exception as e:
            return {'predictions': [{'values': [repr(e)]}]}



In [38]:
%%bash

gzip -c score_fun_git.py > score_fun_git.py.gz 

In [39]:
%%bash --out attachment_id

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@score_fun_git.py.gz" \
    "$DATAPLATFORM_URL/ml/v4/functions/$FUNCTION_ID/code?space_id=$SPACE_ID&version=2023-10-26"\
    | grep "attachment_id" | awk -F '"' '{ print $4 }'


In [40]:
%env ATTACHMENT_ID=$attachment_id

env: ATTACHMENT_ID=27e10622-43ed-46f5-9141-a4332d935783


### python function online deployment

In [41]:
%env FUNCTION_ID

'bcc0ee29-3430-481a-a694-250b98894b5d'

In [49]:
%%bash --out deployment_payload

DEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "Py Function deployment", "description": "This is description","online": {},"asset": {"id": "'"$FUNCTION_ID"'"}}'
echo $DEPLOYMENT_PAYLOAD

In [50]:
JSON(loads(deployment_payload))

<IPython.core.display.JSON object>

In [51]:
%env DEPLOYMENT_PAYLOAD=$deployment_payload

env: DEPLOYMENT_PAYLOAD={"space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31","name": "Py Function deployment", "description": "This is description","online": {},"asset": {"id": "bcc0ee29-3430-481a-a694-250b98894b5d"}}


In [52]:
%%bash --out deployment_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$DEPLOYMENT_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 3p

In [53]:
%env DEPLOYMENT_ID=$deployment_id

env: DEPLOYMENT_ID=Deprecated


#### Score the function/prediction

In [54]:
%%bash

curl -sk -X POST "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID/predictions?version=2023-10-30"\
    --header 'Content-Type: application/json' \
    --header 'Accept: application/json' \
    --header "Authorization:Bearer  $TOKEN" \
    --data-raw '{"input_data":[{"values":[[3]]}]}' 


{"trace":"068181f4d070e0afc3a4049cb833c357","errors":[{"code":"deployment_does_not_exist","message":"Deployment with id 'e783ecb4-a3db-4e42-bd38-df33a8317902' does not exist. Re-try with a valid deployment id."}]}

### python function batch deployment

In [63]:
%%bash --out batch_deployment_payload

BATCH_DFEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "python batch func dep","description": "This is description","batch": {}, "hardware_spec": {"name": "S"},"asset": {"id": "'"$FUNCTION_ID"'"}}'
echo $BATCH_DFEPLOYMENT_PAYLOAD 

In [64]:
JSON(loads(batch_deployment_payload))

<IPython.core.display.JSON object>

In [65]:
%env BATCH_DFEPLOYMENT_PAYLOAD=$batch_deployment_payload

env: BATCH_DFEPLOYMENT_PAYLOAD={"space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31","name": "python batch func dep","description": "This is description","batch": {}, "hardware_spec": {"name": "S"},"asset": {"id": "bcc0ee29-3430-481a-a694-250b98894b5d"}}


In [66]:
%%bash --out func_deployment_id_batch

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$BATCH_DFEPLOYMENT_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments?version=2023-10-30" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 3p


In [67]:
%env FUNC_DEPLOYMENT_ID_BATCH=$func_deployment_id_batch

env: FUNC_DEPLOYMENT_ID_BATCH=7c0ddb07-49d6-4eeb-b1cc-01374d5ee989


#### Score the batch deployment

#### create a job for batch dep

In [95]:
%%bash --out job_payload

JOB_PAYLOAD='{"name": "py func batch job", "space_id": "'"$SPACE_ID"'","deployment": {"id": "'"$FUNC_DEPLOYMENT_ID_BATCH"'"}, "hardware_spec": {"name": "S"},"scoring":{"input_data":[{"values":[[3]]}]}}'
echo $JOB_PAYLOAD 


In [96]:
JSON(loads(job_payload))

<IPython.core.display.JSON object>

In [97]:
%env JOB_PAYLOAD=$job_payload

env: JOB_PAYLOAD={"name": "py func batch job", "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31","deployment": {"id": "7c0ddb07-49d6-4eeb-b1cc-01374d5ee989"}, "hardware_spec": {"name": "S"},"scoring":{"input_data":[{"values":[[3]]}]}}


In [115]:
%%bash 

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$JOB_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployment_jobs?version=2023-10-30" 


{
  "entity": {
    "deployment": {
      "id": "7c0ddb07-49d6-4eeb-b1cc-01374d5ee989"
    },
    "hardware_spec": {
      "name": "S"
    },
    "scoring": {
      "input_data": [{
        "values": [[3]]
      }],
      "status": {
        "state": "queued"
      }
    }
  },
  "metadata": {
    "created_at": "2023-10-31T15:15:11.275Z",
    "id": "b7e8cdfd-0f25-4972-811b-0b37979aa389",
    "name": "py func batch job",
    "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
  }
}

#### list all batch jobs

In [117]:
%%bash
curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployment_jobs?space_id=$SPACE_ID&version=2020-08-01" 

{
  "resources": [{
    "entity": {
      "deployment": {
        "id": "7c0ddb07-49d6-4eeb-b1cc-01374d5ee989"
      },
      "hardware_spec": {
        "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b"
      },
      "platform_job": {
        "job_id": "58ac5da3-eda2-4130-854d-9c7fab86e4ad",
        "run_id": "5bfd3a7e-d458-42f4-8f16-7d10425686ff"
      },
      "scoring": {
        "input_data": [{
          "values": [[3]]
        }],
        "status": {
          "completed_at": "2023-10-31T15:17:55.072Z",
          "failure": {
            "trace": "9df53cddbbeb43c76f3d5f847bdc63a9",
            "errors": [{
              "code": "low_on_resources",
              "message": "Unable to run this job using the specified hardware specification due to insufficient server resources. Change to a smaller configuration or increase available server resources and try again.",
              "more_info": "The runtime failed to start due to 'insufficient resources'"
            }]
          },
     

#### get job details

In [120]:
%env JOB_ID=b7e8cdfd-0f25-4972-811b-0b37979aa389

env: JOB_ID=b7e8cdfd-0f25-4972-811b-0b37979aa389


In [121]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployment_jobs/$JOB_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "deployment": {
            "id": "7c0ddb07-49d6-4eeb-b1cc-01374d5ee989"
        },
        "hardware_spec": {
            "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b"
        },
        "platform_job": {
            "job_id": "58ac5da3-eda2-4130-854d-9c7fab86e4ad",
            "run_id": "0e55f8d8-3877-46c6-8ac3-d9a5dfa395c7"
        },
        "scoring": {
            "input_data": [
                {
                    "values": [
                        [
                            3
                        ]
                    ]
                }
            ],
            "status": {
                "completed_at": "2023-10-31T15:18:50.521Z",
                "failure": {
                    "trace": "e2f3ba55cbcb6ba0c33a5794c1adccd9",
                    "errors": [
                        {
                            "code": "low_on_resources",
                            "message": "Unable to run this job using the specified hardware specification d